In [ ]:
import math
import numpy as np
import cv2
from scipy import ndimage
from matplotlib import pyplot as plt

In [ ]:
def detect_points_with_masks(frame, threshold=0.001, min_pixels=4):
    """
    フレームから白い点を検出し、それぞれの点に対応するマスク（輝度分布）を取得する。
    
    Parameters:
        frame (ndarray): 入力フレーム（2D配列）。
        threshold (float): 輝度の閾値。
        min_pixels (int): 点と見なすための最小ピクセル数。
    
    Returns:
        centers (ndarray): 各点の重心座標の配列（形状: (N, 2)）。
        masks (list of ndarray): 各点のマスク（輝度分布を含む、形状: (H, W)）。
    """
    # 閾値処理
    binary = (frame >= threshold).astype(np.uint8)
    
    # 点のラベリング
    labeled, num_features = ndimage.label(binary)
    
    centers = []
    masks = []
    
    for i in range(1, num_features + 1):
        mask = (labeled == i).astype(np.float32)
        if np.sum(mask) < min_pixels:
            continue  # 小さすぎる点は無視
        
        # 元のフレームの輝度をマスク
        mask_with_brightness = mask * frame
        
        # 重心の計算
        center = ndimage.center_of_mass(mask)
        centers.append(center)
        masks.append(mask_with_brightness)
    
    centers = np.array(centers)  # 形状: (N, 2)
    return centers, masks

def compute_motion_vectors(prev_centers, next_centers, max_distance=10):
    """
    各点の動きベクトルを計算する。
    最も近い点を対応付けとして動きベクトルを計算する。
    
    Parameters:
        prev_centers (ndarray): 前のフレームの点の重心座標（形状: (N, 2)）。
        next_centers (ndarray): 次のフレームの点の重心座標（形状: (M, 2)）。
        max_distance (float): 対応付けを許容する最大距離。
    
    Returns:
        motion_vectors (ndarray): 各前フレームの点に対応する動きベクトル（形状: (N, 2)）。
    """
    motion_vectors = []
    used_next = set()
    
    for p in prev_centers:
        distances = np.linalg.norm(next_centers - p, axis=1)
        min_idx = np.argmin(distances)
        min_dist = distances[min_idx]
        
        if min_dist <= max_distance and min_idx not in used_next:
            used_next.add(min_idx)
            vector = next_centers[min_idx] - p  # (dy, dx)
        else:
            vector = np.array([0.0, 0.0])  # 対応が見つからない場合は動かない
        motion_vectors.append(vector)
    
    return np.array(motion_vectors)  # 形状: (N, 2)

def interpolate_masks(current_masks, next_masks, motion_vectors, alpha, frame_shape):
    """
    各点のマスクを補間位置に移動させて新しいマスクを生成する。
    各点について、前フレームのマスクを1/2動きベクトル分移動させ、
    次フレームのマスクを-1/2動きベクトル分移動させたものをブレンドする。
    
    Parameters:
        current_masks (list of ndarray): 前のフレームのマスク（輝度分布）。
        next_masks (list of ndarray): 次のフレームのマスク（輝度分布）。
        motion_vectors (ndarray): 各点の動きベクトル（形状: (N, 2)）。
        alpha (float): 補間係数（通常は0.5）。
        frame_shape (tuple): 補間フレームの形状（高さ, 幅）。
    
    Returns:
        interp_frame (ndarray): 補間フレーム（2D配列）。
    """
    interp_frame = np.zeros(frame_shape, dtype=np.float32)
    num_points = len(current_masks)
    
    for i in range(num_points):
        vector = motion_vectors[i]
        half_vector = vector * 0.5
        
        # 前フレームのマスクを1/2動きベクトル分移動
        M_current = np.float32([[1, 0, half_vector[1]], [0, 1, half_vector[0]]])  # [ [1, 0, dx], [0, 1, dy] ]
        shifted_current = cv2.warpAffine(current_masks[i], M_current, (frame_shape[1], frame_shape[0]),
                                            flags=cv2.INTER_LINEAR, borderValue=0)
        
        # 次フレームのマスクを-1/2動きベクトル分移動
        M_next = np.float32([[1, 0, -half_vector[1]], [0, 1, -half_vector[0]]])
        shifted_next = cv2.warpAffine(next_masks[i], M_next, (frame_shape[1], frame_shape[0]),
                                            flags=cv2.INTER_LINEAR, borderValue=0)
        
        # マスクのブレンド（平均）
        blended_mask = (shifted_current + shifted_next) / 2.0
        
        # 補間フレームに加算
        interp_frame += blended_mask
    
    # クランプ
    interp_frame = np.clip(interp_frame, 0.0, 1.0)
    return interp_frame

def interpolate_frames(frames, num_interpolations=1):
    """
    フレーム補間を行う。
    各フレーム間に指定された数の補間フレームを挿入する。
    
    Parameters:
        frames (ndarray): 元のフレーム配列（形状: (num_frames, H, W)）。
        num_interpolations (int): 各フレーム間に挿入する補間フレームの数。
    
    Returns:
        interpolated (ndarray): 補間後のフレーム配列（形状: new_num_frames, H, W）。
    """
    interpolated = []
    num_frames, height, width = frames.shape
    
    for t in range(num_frames - 1):
        current_frame = frames[t]
        next_frame = frames[t + 1]
        
        # 点の検出とマスクの取得
        current_centers, current_masks = detect_points_with_masks(current_frame)
        next_centers, next_masks = detect_points_with_masks(next_frame)
        
        # 動きベクトルの計算
        motion_vectors = compute_motion_vectors(current_centers, next_centers)
        
        interpolated.append(current_frame)
        
        for k in range(1, num_interpolations + 1):
            # 補間係数（通常は0.5）
            alpha = k / (num_interpolations + 1)
            
            # 補間フレームの生成
            interp_frame = interpolate_masks(current_masks, next_masks, motion_vectors, alpha, (height, width))
            
            interpolated.append(interp_frame)
    
    interpolated.append(frames[-1])
    return np.array(interpolated)

In [59]:
# DATA_PATH = "../data"
DATA_PATH = "../../OneDrive - m.titech.ac.jp/Lab/data"
H_SETTING = "hadamard_FISTA_p-5_lmd-1_m-128"
frames = np.load(f"{DATA_PATH}/241022/systemMatrix/H_tensor_{H_SETTING}.npy")

In [ ]:
# フレーム補間
interpolated_frames = interpolate_frames(frames, num_interpolations=1)
print(f"Original frames: {frames.shape}")
print(f"Interpolated frames: {interpolated_frames.shape}")

In [ ]:
# 表示したいフレームの範囲を設定
start_index = 32
end_index = 47
frames_to_display = interpolated_frames[start_index:end_index + 1]

num_frames = len(frames_to_display)
cols = 4  # 列数を設定（必要に応じて変更可能）
rows = math.ceil(num_frames / cols)  # 行数を自動計算

# サブプロットを作成
fig, axs = plt.subplots(rows, cols, figsize=(5 * cols, 5 * rows))

# サブプロットの軸を1次元にフラット化してループ処理
for idx, ax in enumerate(axs.flat):
    if idx < num_frames:
        frame = frames_to_display[idx]
        img = ax.imshow(frame, cmap='gray')
        plt.colorbar(img, ax=ax)
        ax.set_title(f'Frame {start_index + idx}')
    else:
        ax.axis('off')  # 使用しないサブプロットを非表示に

# import imageio
# frames_to_save = ((interpolated_frames / 0.02) * 255).astype(np.uint8)
# imageio.mimsave('interpolated_video.gif', frames_to_save, fps=30)
# print("Interpolated video saved as 'interpolated_video.gif'.")